In [ ]:
############################################################################################
# IMPORTS
############################################################################################
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay


import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
#import tensorflow.keras as keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers


from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical



from IPython import display
from PIL import Image


import pathlib
import shutil
import tempfile
import concurrent


import json
import glob


import cv2




print(tf.__version__)
print(tf.config.list_physical_devices())
############################################################################################

In [ ]:
############################################################################################
# 1. CONSTANTS - PATHS
############################################################################################

DATA_FS251 = './data/iFood_2019'
CLASSES_FILE_NAME = '/formated_annot/classes_formated.csv'

TRAIN_INFO = '/annot/train_info.csv'
VAL_INFO = '/annot/val_info.csv'
TEST_INFO = '/annot/test_info.csv'

TRAIN_PICS_PATH = './data/iFood_2019/train_set/'
TEST_PICS_PATH = './data/iFood_2019/test_set/'
VAL_PICS_PATH = './data/iFood_2019/val_set/'

MODELS = './models/'

SEED = 111

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [ ]:
############################################################################################
# IMPORTING DATA
############################################################################################
df_classes = pd.read_csv(DATA_FS251 + CLASSES_FILE_NAME)
df_train = pd.read_csv(DATA_FS251 + TRAIN_INFO, names=['file_name', 'class_num'])
df_validate = pd.read_csv(DATA_FS251 + VAL_INFO, names=['file_name', 'class_num'])
df_test = pd.read_csv(DATA_FS251 + TEST_INFO, names=['file_name'])

In [ ]:
############################################################################################
# 2. CONSTANTS - MODEL
############################################################################################
training_history = dict()

N_TRAIN = len(df_train.iloc[:, 0])
EPOCHS = 30
BATCH_SIZE = 1              # 8 fit into GPU RAM, 64 fit into system RAM
RESIZE_TO = (336, 336)
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE

In [ ]:
############################################################################################
# FUNCTIONS, DEFFINITIONS
############################################################################################

class CRelu(tf.keras.Layer):
    def __init__(self, axis=-1, **kwargs):
        self.axis = axis 
        super(CRelu, self).__init__(**kwargs)

    def build(self, input_shape):
        super(CRelu, self).build(input_shape)

    def call(self, x):
        x = tf.nn.crelu(x, axis=self.axis)
        return x

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[-1] = output_shape[-1] * 2
        output_shape = tuple(output_shape)
        return output_shape

    def get_config(self, input_shape):
        config = {'axis': self.axis, }
        base_config = super(CReLU, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
def img_label_import(df, path_to_pics, oh_encoded=True):
    pics = []
    lab = []
    for iter in range(df.shape[0]):
        pict = cv2.imread(os.path.join(path_to_pics, str(df['file_name'].iloc[iter]) )  )
        pics.append(cv2.resize(pict,RESIZE_TO))
        lab.append(df['class_num'].iloc[iter])
    if oh_encoded:
          return np.array(pics).astype('uint8'), np.array(pd.get_dummies(lab).values).astype('uint8')
    else:
          return np.array(test_pics).astype('uint8'), np.array(test_lab).astype('uint8')

In [ ]:
train_pics = []
train_lab = []
test_pics = []
test_lab = []


df_train_sample = df_train.sample(n=20000, random_state=SEED)
df_validate_sample = df_validate.sample(n=4000, random_state=SEED)





#train_pics, train_lab = img_label_import(df=df_train_sample, path_to_pics=TRAIN_PICS_PATH, oh_encoded=True)
#test_pics, test_lab = img_label_import(df=df_validate_sample, path_to_pics=VAL_PICS_PATH, oh_encoded=False)


for iter in range(df_train_sample.shape[0]):
        pict = cv2.imread(os.path.join(TRAIN_PICS_PATH, str(df_train_sample['file_name'].iloc[iter]) )  )
        train_pics.append(cv2.resize(pict,RESIZE_TO))
        train_lab.append(df_train_sample['class_num'].iloc[iter])
train_pics = np.array(train_pics).astype('uint8')
train_lab = np.array(pd.get_dummies(train_lab).values).astype('uint8')




for iter in range(df_validate_sample.shape[0]):
        pict = cv2.imread(os.path.join(VAL_PICS_PATH, str(df_validate_sample['file_name'].iloc[iter]) ), 3 )
        test_pics.append(cv2.resize(pict,RESIZE_TO))
        test_lab.append(df_validate_sample['class_num'].iloc[iter])
test_pics = np.array(test_pics).astype('uint8')
test_lab = np.array(test_lab).astype('uint8')

In [ ]:
#x_train,x_val,y_train,y_val = train_test_split(train_pics,train_lab, train_size=0.8, random_state=SEED)
del train_pics, train_lab

In [ ]:
plt.imshow(train_pics[40])
#plt.title(y_train[40])
plt.show()
x_train.shape

In [ ]:
############################################################################################
# DEFINING THE MODEL
############################################################################################
tf.keras.backend.clear_session()
model_supclass = tf.keras.models.Sequential(layers=[
# Zero's layer - resizing:
        tf.keras.layers.InputLayer(shape=(336,336,3)),
        tf.keras.layers.Resizing(
                height = 336,
                width = 336,
                interpolation='bilinear',
                crop_to_aspect_ratio=False,
                pad_to_aspect_ratio=True,
                fill_mode='constant',
                fill_value=0.0,
                data_format=None
        ),
        tf.keras.layers.Rescaling(1./255),
#        tf.keras.layers.Normalization(),
# First conv. layers set 256p:
        tf.keras.layers.Conv2D(
                filters=32,
                kernel_size = (2,2),
                strides=(1, 1),
                padding='same',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
        tf.keras.layers.Conv2D(
                filters=32,
                kernel_size = (2,2),
                strides=(1, 1),
                padding='same',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
# Second conv. layers set 256->128:
        tf.keras.layers.MaxPooling2D(
                pool_size=(2, 2),
                strides=2,
                padding='valid',
                data_format=None,
                name='256_to_128_MaxPooling2D'
        ),
        tf.keras.layers.Conv2D(
                filters=16,
                kernel_size = (4,4),
                strides=(1, 1),
                padding='same',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
# 3rd conv. layers set 128->64:
        tf.keras.layers.MaxPooling2D(
                pool_size=(2, 2),
                strides=2,
                padding='valid',
                data_format=None,
                name='128_to_64_MaxPooling2D'
        ),
        tf.keras.layers.Conv2D(
                filters=8,
                kernel_size = (2,2),
                strides=(1, 1),
                padding='same',
                data_format=None,
                dilation_rate=(1, 1),
                groups=1,
                activation='relu',
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None
        ),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=512,activation='relu'),
        tf.keras.layers.Dense(units=251,activation='sigmoid')
])

############################################################################################
# COMPILING
############################################################################################
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.005, #0.000005
    decay_steps=STEPS_PER_EPOCH*100, #100
    decay_rate=1,
    staircase=False)

model_supclass.compile(
#        optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),
#        optimizer = tf.keras.optimizers.Adam(),
        optimizer = tf.keras.optimizers.Adagrad(),
#        optimizer = tf.keras.optimizers.Adadelta(),
#        optimizer='rmsprop',
        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False,label_smoothing=0,axis=-1,reduction='sum_over_batch_size'),
#        loss = tf.keras.losses.SparseCategoricalCrossentropy()
#        metrics = [tf.keras.metrics.CategoricalCrossentropy(from_logits=False, label_smoothing=0.2, axis=-1),
#                   tf.keras.metrics.F1Score(average=None, threshold=0.5, dtype=None),
#                   tf.keras.metrics.CategoricalAccuracy()]
        metrics = [tf.keras.metrics.Accuracy()]
#        metrics = [tf.keras.metrics.F1Score(average='micro', threshold=0.5),tf.keras.metrics.CategoricalAccuracy()]
        )

model_supclass.summary()

In [ ]:
############################################################################################
# FITTING
############################################################################################
with tf.device('/gpu:0'):
    model_history = model_supclass.fit(
        x_train,
        y_train,
        epochs = EPOCHS, 
        batch_size = BATCH_SIZE,
#        callbacks = tf.keras.callbacks.EarlyStopping(
#                            monitor='val_loss',
#                            min_delta=0.01,
#                            patience=1,
#                            verbose=1,
#                            mode='auto',
#                            baseline=None,
#                            restore_best_weights=True,
#                            start_from_epoch=0),
        validation_split = 0.2
    )

In [ ]:
############################################################################################
# SAVING MODEL
############################################################################################

os.makedirs( os.path.dirname(MODELS), exist_ok=True)
model_supclass.save(MODELS + "custom_251_n32_adagrad_b1_e1_v0_2_rawinput.keras")
model_supclass_loaded = load_model(MODELS + "custom_251_n32_adagrad_b1_e1_v0_2_rawinput.keras")
model_supclass_loaded.summary()

In [ ]:
############################################################################################
# EVALUATION
############################################################################################

plt.legend(bbox_to_anchor = [1, 1.02])
plt.plot(model_history.history['accuracy'],label='train_accuracy')
plt.plot(model_history.history['val_accuracy'],label='val_accuracy')
plt.xlabel('epochs')
plt.ylabel('categorical_accuracy')
plt.legend()

# Print out the score
score = model_supclass.evaluate(x_val, y_val , batch_size=1, verbose=1)
print(score, model_supclass.metrics_names)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix


plt.figure(figsize = (30,30))
cm = confusion_matrix(y_true= y_test, y_pred= )
#disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=np.unique(y_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

In [ ]:
STOP_HERE
BELOW_IS_THE_RESIDUAL_CODE_FOR_HISTORY_PURPOSE

In [ ]:
############################################################################################
# import image files
############################################################################################

train_pics, val_pics = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PICS_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
    shuffle=False,
    seed=SEED,
    validation_split=0.2,
    subset='both',
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format=None,
    verbose=True
)
test_pics = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_PICS_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
    shuffle=False,
    seed=SEED,
    validation_split=None,
    subset=None,
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format=None,
    verbose=True
)
true_test_pics = tf.keras.preprocessing.image_dataset_from_directory(
    TEST_PICS_PATH,
    labels=None,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
    shuffle=False,
    seed=SEED,
    validation_split=None,
    subset=None,
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format=None,
    verbose=True
)

In [ ]:
#plt.figure(figsize=(15, 15))
#for images, labels in train_08_pics.take(1):
#    for i in range(9):
#        ax = plt.subplot(3, 3, i + 1)
#        plt.imshow(images[i].numpy().astype("uint8"))
#        plt.title(class_names[i])
#        plt.axis("off")

In [ ]:
############################################################################################
# COMPILING
############################################################################################
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.005, #0.000005
    decay_steps=STEPS_PER_EPOCH*100, #100
    decay_rate=1,
    staircase=False)

model_supclass.compile(
#        optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),
        optimizer = tf.keras.optimizers.Adam(),
#        optimizer = tf.keras.optimizers.Adalta(),
#        optimizer='rmsprop',
        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False,label_smoothing=0.2,axis=-1,reduction='sum_over_batch_size'),
#        metrics = [tf.keras.metrics.CategoricalCrossentropy(from_logits=False, label_smoothing=0.2, axis=-1),
#                   tf.keras.metrics.F1Score(average=None, threshold=0.5, dtype=None),
#                   tf.keras.metrics.CategoricalAccuracy()]
        metrics = [tf.keras.metrics.CategoricalAccuracy()]
#        metrics = [tf.keras.metrics.F1Score(average='micro', threshold=0.5),tf.keras.metrics.CategoricalAccuracy()]
        )

In [ ]:
############################################################################################
# FITTING
############################################################################################
with tf.device('/gpu:0'):
    model_history = model_supclass.fit(
        train_pics,
        epochs = EPOCHS, 
        batch_size = BATCH_SIZE,
#        callbacks = tf.keras.callbacks.EarlyStopping(
#                            monitor='val_loss',
#                            min_delta=0.01,
#                            patience=1,
#                            verbose=1,
#                            mode='auto',
#                            baseline=None,
#                            restore_best_weights=True,
#                            start_from_epoch=0),
        validation_data = val_pics
    )

In [ ]:
############################################################################################
# SAVING MODEL
############################################################################################

os.makedirs( os.path.dirname(MODELS), exist_ok=True)
model_supclass.save(MODELS + "custom_251_n32_adadelta_b4_e1_wo_norm_v0_1.keras")
model_supclass_loaded = load_model(MODELS + "custom_251_n32_adadelta_b4_e1_wo_norm_v0_1.keras")
model_supclass_loaded.summary()



In [ ]:
model_history.history['categorical_accuracy']

In [ ]:
############################################################################################
# EVALUATION
############################################################################################

plt.legend(bbox_to_anchor = [1, 1.02])
plt.plot(model_history.history['categorical_accuracy'],label='train_categorical_accuracy')
plt.plot(model_history.history['val_categorical_accuracy'],label='val_categorical_accuracy')
plt.xlabel('epochs')
plt.ylabel('categorical_accuracy')
plt.legend()

# Print out the score
score = model_supclass_loaded.evaluate(val_pics , batch_size=4, verbose=1)
print(score, model_supclass_loaded.metrics_names)

In [ ]:
############################################################################################
# PREDICTION
############################################################################################

y_test_pred = np.array([])
y_test =  np.array([])
for x, y in test_pics:
  y_test_pred = np.concatenate([y_test_pred, np.argmax(model_supclass.predict(x, verbose=0), axis = -1)])
  y_test = np.concatenate([y_test, np.argmax(y.numpy(), axis=-1)])

In [ ]:
############################################################################################
# PREDICTION - CONFMATRIX
############################################################################################

from sklearn.metrics import ConfusionMatrixDisplay
plt.figure(figsize = (30,30))
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true= y_test, y_pred=y_test_pred)
#disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=np.unique(y_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()




